In [1]:
using InteractiveUtils, Pkg
versioninfo(); Pkg.status()

Julia Version 1.6.7
Commit 3b76b25b64 (2022-07-19 15:11 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, cascadelake)
Environment:
  JULIA_NUM_THREADS = 4
      Status `/mnt/efs/kai/projects/AdvancedHMC.jl/research/tests/Project.toml`
  [c75e803d] AdaptiveRejectionSampling v0.1.1
  [0bf59076] AdvancedHMC v0.4.0 `../..#kx/rhmc-draft`
  [863f3e99] Comonicon v1.0.4
  [31c24e10] Distributions v0.24.18
  [366bfd00] DynamicPPL v0.10.20
  [6a86dc24] FiniteDiff v2.17.0
  [6d524b87] MCMCDebugging v0.2.0
  [91a5bcdd] Plots v1.38.6
  [d330b81b] PyPlot v2.11.1
  [e0db7c4e] ReTest v0.3.2
  [37e2e3b7] ReverseDiff v1.9.0
  [8a639fad] VecTargets v0.2.0 `https://github.com/xukai92/VecTargets.jl#main`


In [2]:
using LinearAlgebra, FiniteDiff, VecTargets

In [3]:
target = Funnel()

ℓπ = x -> logpdf(target, x)
neg_ℓπ = x -> -logpdf(target, x)

H = x -> VecTargets.gen_hess(ℓπ, x)(x)[3]
G = x -> VecTargets.gen_hess(neg_ℓπ, x)(x)[3]

#7 (generic function with 1 method)

In [4]:
xt = [0.0, 0.0] # x test

@show H(xt) G(xt)

# Hamiltonian_partial(x) = begin x
#     lad, s = logabsdet(G(x))
#     lad * s
# end # WRONG implementation of the second term of (13)
      # `s` returned is the sign of `det(G)`, not the whole thing
Hamiltonian_partial(x) = begin x
    logdet(G(x))
end # second term of (13)

@show FiniteDiff.finite_difference_gradient(Hamiltonian_partial, xt)

Jt = FiniteDiff.finite_difference_jacobian(G, xt)
@show [tr(inv(G(xt)) * Jt[1:2,1:2]), tr(inv(G(xt)) * Jt[3:4,1:2])]

H(xt) = [-0.1111111111111111 0.0; 0.0 -1.0]
G(xt) = [0.1111111111111111 0.0; 0.0 1.0]
FiniteDiff.finite_difference_gradient(Hamiltonian_partial, xt) = [-0.9999999999502168, 0.0]
[tr(inv(G(xt)) * Jt[1:2, 1:2]), tr(inv(G(xt)) * Jt[3:4, 1:2])] = [-0.9999999664723873, 0.0]


2-element Vector{Float64}:
 -0.9999999664723873
  0.0